## Transform Order data ( JSON Object ) - Explode Arrays
1. Access element from the JSON object
1. De-Duplicate array elements
1. Explode Array
1. Write the transformed data to Silver schema

In [0]:
select * from gizmobox.silver.orders_json

#### Access element from the JSON object


In [0]:
select 
       json_value.order_id,
       json_value.order_status,
       json_value.payment_method,
       json_value.total_amount,
       json_value.transaction_timestamp,
       json_value.customer_id,
       json_value.items
 from gizmobox.silver.orders_json

#### Check for Duplicate array elements


In [0]:
select 
  size(json_value.items) as count,
  size(array_distinct(json_value.items)),
  case when size(json_value.items) <> size(array_distinct(json_value.items)) then 'Have Duplicate'
       else "No duplicate" end as Dup_Check,
  json_value.items
 from gizmobox.silver.orders_json group by json_value.items


#### De-Duplicate array elements

In [0]:
select 
       json_value.order_id,
       json_value.order_status,
       json_value.payment_method,
       json_value.total_amount,
       json_value.transaction_timestamp,
       json_value.customer_id,
       array_distinct(json_value.items) as items
 from gizmobox.silver.orders_json

#### Explode Array


In [0]:
create or replace temp view v_orders_explode
as
select 
       json_value.order_id,
       json_value.order_status,
       json_value.payment_method,
       json_value.total_amount,
       json_value.transaction_timestamp,
       json_value.customer_id,
       explode(array_distinct(json_value.items)) as item
 from gizmobox.silver.orders_json

In [0]:
select * from v_orders_explode

In [0]:
select 
       order_id,
       order_status,
       payment_method,
       total_amount,
       transaction_timestamp,
       customer_id,
       item.category,
       item.details.brand,
       item.details.color,
       item.item_id,
       item.name,
       item.price,
       item.quantity
from v_orders_explode

#### Write the transformed data to Silver schema

In [0]:
create table gizmobox.silver.orders
as
select 
       order_id,
       order_status,
       payment_method,
       total_amount,
       transaction_timestamp,
       customer_id,
       item.category,
       item.details.brand,
       item.details.color,
       item.item_id,
       item.name,
       item.price,
       item.quantity
from v_orders_explode


In [0]:
select * from gizmobox.silver.orders